# 1. Тренировка агрегации меток асессоров

Грамотно формализовать и агрегировать метки асессоров - попловина успеха измерения качества в любой задаче. В текущем примере мы попробуем сравнить два популярных метода агрегации оценок асессоров, которые ранее были выставлены в соотвествии с какой-то инструкцией.

Итак, необходимо считать данные, агрегировать метки асессоров при попощи наивного и продвинутого алгоритмов, а затем устроить "стресс-тестирование" этим методам агрегации. Удачи!

Подробнеее:

https://github.com/ahndana/nlp-final-snli (https://nlp.stanford.edu/projects/snli/)

https://habr.com/ru/companies/mts_ai/articles/747024/

In [ ]:
# !pip install crowd-kit

In [ ]:
import pandas as pd
from crowdkit.aggregation import MajorityVote, DawidSkene

In [ ]:
df = pd.read_csv('snli_1.0_dev.txt', sep = '\t')
df = df.dropna()
df = df[df.gold_label != "-"]
labels = df[df.columns[-5:].to_list()]
labels = labels.reset_index()
df = df[["sentence1", "sentence2", "gold_label"]]
df.reset_index(drop=True, inplace=True)
melted_labels = labels.melt(id_vars="index", )
melted_labels.columns = ["task", "worker", "label"]

In [ ]:
df

In [ ]:
melted_labels.head()

## На исходных данных

### умный метод - девид скин

In [ ]:
model = DawidSkene(n_iter=100)
agg = model.fit_predict_proba(melted_labels)
model.errors_

In [ ]:
agg_labels = pd.DataFrame([agg.idxmax(axis=1), agg.max(axis=1)]).T.reset_index(drop=True)
df = pd.concat([df, agg_labels], axis=1, ignore_index=True)
df.columns = ["sent1", "sent2", "gold_label", "agg_label", "proba"]
df.proba.hist()

In [ ]:
agg_labels.to_csv('agg_labels_devid_5.csv')

### метод "моды"

In [ ]:
selected_melted_labels = melted_labels[melted_labels.worker.isin(["label1", "label4", "label5"])]
another_model = DawidSkene()
agg_ds = another_model.fit_predict_proba(selected_melted_labels)

mv_model = MajorityVote()
agg_mv = mv_model.fit_predict_proba(selected_melted_labels)

agg_labels_ds = pd.DataFrame([agg_ds.idxmax(axis=1), agg_ds.max(axis=1)]).T.reset_index(drop=True)
agg_labels_mv = pd.DataFrame([agg_mv.idxmax(axis=1), agg_mv.max(axis=1)]).T.reset_index(drop=True)

df = pd.concat([df, agg_labels_ds, agg_labels_mv], axis=1, ignore_index=True)
df.columns = ["sent1", "sent2", "gold_label", "agg_label_5", "proba_5", "agg_label_3_ds", "proba_3_ds", "agg_label_3_mv", "proba_3_mv"]

diff_ds = df[df.gold_label != df.agg_label_3_ds]
diff_mv = df[df.gold_label != df.agg_label_3_mv]
len(diff_ds), len(diff_mv)

In [ ]:
# на всех экспертах применим метод "моды"
mv_model = MajorityVote()
agg_mv = mv_model.fit_predict_proba(melted_labels)
agg_labels_mv = pd.DataFrame([agg_mv.idxmax(axis=1), agg_mv.max(axis=1)]).T.reset_index(drop=True)
agg_labels_mv.to_csv('agg_labels_moda_5.csv')

In [ ]:
melted_labels

## Модифицируем данные - добавляем еще асессоров - рандомайзеров

In [ ]:
df = pd.read_csv('snli_1.0_dev.txt', sep = '\t')
df = df.dropna()
df = df[df.gold_label != "-"]
labels = df[df.columns[-5:].to_list()]
labels = labels.reset_index()
df = df[["sentence1", "sentence2", "gold_label"]]
df.reset_index(drop=True, inplace=True)
melted_labels = labels.melt(id_vars="index", )
melted_labels.columns = ["task", "worker", "label"]

In [ ]:
import numpy as np
import random

COUNT_NEW_WORKERS = 10
new_workers = []
for i in range(COUNT_NEW_WORKERS):
    random.seed(42+i)
    new_labels = np.random.choice(melted_labels['label'].unique(), size = melted_labels['task'].nunique() )
    melted_labels_add = pd.DataFrame({'label':new_labels, 'task': melted_labels['task'].unique()})
    melted_labels_add['worker'] = f'label{i+6}'
    new_workers.append(melted_labels_add)
melted_labels = pd.concat([melted_labels]+new_workers)

In [ ]:
melted_labels

### умный метод

In [ ]:
model = DawidSkene(n_iter=100)
agg = model.fit_predict_proba(melted_labels)
model.errors_

In [ ]:
agg_labels = pd.DataFrame([agg.idxmax(axis=1), agg.max(axis=1)]).T.reset_index(drop=True)
df = pd.concat([df, agg_labels], axis=1, ignore_index=True)
df.columns = ["sent1", "sent2", "gold_label", "agg_label", "proba"]
df.proba.hist()

In [ ]:
agg_labels.to_csv('agg_labels_devid_15.csv')

### метод "моды"

In [ ]:
# на всех экспертах применим метод "моды"
mv_model = MajorityVote()
agg_mv = mv_model.fit_predict_proba(melted_labels)
agg_labels_mv = pd.DataFrame([agg_mv.idxmax(axis=1), agg_mv.max(axis=1)]).T.reset_index(drop=True)
agg_labels_mv.to_csv('agg_labels_moda_15.csv')

## Анализ отличий

In [ ]:
agg_labels_mv = dict()
agg_labels_devid = dict()
agg_labels_devid = pd.read_csv(f'agg_labels_devid_5.csv', index_col = 0)
agg_labels_devid.columns = ['label_devid', 'p_label']
agg_labels_mv = pd.read_csv(f'agg_labels_moda_5.csv', index_col = 0)
agg_labels_mv.columns = ['label_moda', 'p_label']

for NUM_BAD_LABELERS in [1, 5, 10]:
    print("NUM_BAD =", NUM_BAD_LABELERS)
    agg_labels_devid_bad = pd.read_csv(f'agg_labels_devid_{5 + NUM_BAD_LABELERS}.csv', index_col = 0)
    agg_labels_devid_bad.columns = ['label_devid_bad', 'p_label']
    agg_labels_mv_bad = pd.read_csv(f'agg_labels_moda_{5 + NUM_BAD_LABELERS}.csv', index_col = 0)
    agg_labels_mv_bad.columns = ['label_moda_bad', 'p_label']
    
    print("Devid Skeene:")
    devid = pd.concat([agg_labels_devid, agg_labels_devid_bad], axis = 1)
    print(devid.apply(lambda x: x['label_devid']!=x['label_devid_bad'], axis=1).mean())


    print("Majority:")
    major = pd.concat([agg_labels_mv, agg_labels_mv_bad], axis = 1)
    print(major.apply(lambda x: x['label_moda']!=x['label_moda_bad'], axis=1).mean())
    print()

## Итоги 1 части:

|x|Девид Скин|Голосование|
|---|---|---|
| % разлинчых итоговых меток для 1 "паршивой овцы" | ? %| ? %|
| % разлинчых итоговых меток для 5 "паршивых овец" | ? %| ? %|
| % разлинчых итоговых меток для 10 "паршивых овец"|? %| ? %|

На практике, среди размечтиков каждый может ошибиться и разница в "порядочности" при выполнении работы не так заметна, как в текущем примере. Однако данный пример позволяет нагялдно визуализировать важность использования продвинутых алгоритмов, не смотря на простоту освоения "наивных" методов.

# 2. Составить ml-system-design-doc

Берем свой процесс ИЛИ с сайта https://www.evidentlyai.com/ml-system-design любой процесс с тегом NLP

Заполняем https://github.com/IrinaGoloshchapova/ml_system_design_doc_ru?tab=readme-ov-file 

На основе этого определяем гиперпараметры своего ML-решения


# Критерии оценки

1 часть - получены корректные количественные результаты

2 часть - полнота и релевантность описанного документа:
- Полнота: проверяем все ли аспекты решения запланированы к проверке? Все ли необходимые для дальнейшего анализа вводные описаны?
- Релевантность: нет ли лишних проверок? Нет ли лишней или запутывающей информации?
